# TSA Chapter 3: Python Lab: Forecast Accuracy

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch3/TSA_ch3_python_forecast/TSA_ch3_python_forecast.ipynb)

This notebook demonstrates:
- Rolling ARIMA forecast with accuracy metrics: MAE, RMSE, MAPE
- Out-of-sample evaluation.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
ar = np.array([1, -0.6])
ma = np.array([1, 0.3])
stationary = ArmaProcess(ar, ma).generate_sample(nsample=250)
data = 100 + np.cumsum(stationary)  # add offset for MAPE

train_n = 200
forecasts, actuals = [], []

for t in range(train_n, len(data)):
    model = ARIMA(data[:t], order=(1, 1, 1)).fit()
    fc = model.forecast(steps=1)
    forecasts.append(fc.iloc[0] if hasattr(fc, 'iloc') else fc[0])
    actuals.append(data[t])

forecasts = np.array(forecasts)
actuals = np.array(actuals)
errors = actuals - forecasts

mae = np.mean(np.abs(errors))
rmse = np.sqrt(np.mean(errors**2))
mape = np.mean(np.abs(errors / actuals)) * 100

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

axes[0].plot(range(train_n, len(data)), actuals, color='#1A3A6E', label='Actual')
axes[0].plot(range(train_n, len(data)), forecasts, color='#DC3545', linestyle='--', label='Forecast')
axes[0].set_title('1-Step Ahead Forecast', fontweight='bold')
axes[0].legend()

axes[1].bar(range(len(errors)), errors, color='#1A3A6E', alpha=0.7)
axes[1].axhline(0, color='red', linestyle='--')
axes[1].set_title('Forecast Errors', fontweight='bold')

metrics = [mae, rmse, mape]
labels = ['MAE', 'RMSE', 'MAPE (%)']
axes[2].bar(labels, metrics, color=['#1A3A6E', '#DC3545', '#2E7D32'], alpha=0.8)
axes[2].set_title('Accuracy Metrics', fontweight='bold')
for i, v in enumerate(metrics):
    axes[2].text(i, v + 0.01*max(metrics), f'{v:.3f}', ha='center', fontsize=10)

plt.tight_layout()
save_chart(fig, 'sem3_forecast_accuracy')
plt.show()

print(f"Forecast Accuracy:")
print(f"  MAE:  {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAPE: {mape:.4f}%")